In [ ]:
import json
import string
import random 
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# used a dictionary to represent an intents JSON file
data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Hello", "How are you?", "Hi there", "Hi", "Whats up"],
              "responses": ["Hello"]
             },
             {"tag": "age",
              "patterns": ["how old are you?", "when is your birthday?", "when was you born?"],
              "responses": ["I am 24 years old", "I was born in 1996", "My birthday is July 3rd 2022"]
             },
             {"tag": "date",
              "patterns": ["what are you doing this weekend?",
"do you want to hang out some time?", "what are your plans for this week"],
              "responses": ["I am available all week", "I don't have any plans"]
             },
             {"tag": "name",
              "patterns": ["what's your name?", "what are you called?", "who are you?","name?"],
              "responses": ["My name is Omi"]
             },
             {"tag": "goodbye",
              "patterns": [ "bye", "stop", "see ya", "adios", "Thx"],
              "responses": ["It was nice speaking to you. See you later."]
             },
             {"tag": "loan",
              "patterns": [ "loan?", "loan available?", "What am i eligible for?", "loans", "farmer loan"],
              "responses": ["I am searching for top loan plans suited for you."]
             },
             {"tag": "loan",
              "patterns": [ "loan?", "loan available?", "What am i eligible for?", "loans", "farmer loan"],
              "responses": ["I am searching for top loan plans suited for you."]
             }
]}

In [ ]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("Plays")

'Plays'

In [ ]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [ ]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5888      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
Total params: 14,534
Trainable params: 14,534
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
1/1 [==============================

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5888      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
Total params: 14,534
Trainable params: 14,534
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 1.4 MB 25.5 MB/s 
     |████████████████████████████████| 65 kB 3.9 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=f56a56f5f13b22700a62ecf4de96d4c10f2283c4875dece4622a900aadcf6f5f
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [ ]:
import googletrans
translator = googletrans.Translator()
a="நீங்கள் எப்படி இருக்கிறீர்கள்"
translated = translator.translate(a,src='ta',dest='en')
print(translated.text)

How are you


Python speech to text translation

In [ ]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 1.3 MB/s 


In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (295 kB/s)
Selecting p

In [ ]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
import soundfile as sf
import speech_recognition as sr 
def audio_input():
  audio, tsr = get_audio()
  sf.write('new_file.wav', audio, 44100)
  Audio_file = "/content/new_file.wav"
  r = sr.Recognizer()
  with sr.AudioFile(Audio_file) as source:
      audio = r.record(source)  # read the entire audio file
  try:
    tn_text = r.recognize_google(audio,language="ta-IN")
    print(tn_text)
    a = translator.translate(tn_text,src='ta',dest='en')
  except sr.UnknownValueError:
      a = "Google Speech Recognition could not understand audio"
      print(translator.translate(a,src='en',dest='ta').text)
      return 0
  return a.text.strip()

In [ ]:
# running the chatbot
#- *- coding: utf- 8 - *-
import re
def sendmessage(message):
  translated = translator.translate(message,dest='en')
  #print(translated.text)
  intents = pred_class(message, words, classes)
  result = get_response(intents, data)
  translated = translator.translate(result,src='en',dest='ta')
  ret = translated.text
  return ret

Setup function for train ticket booking

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
def func(sen):
  #Name Entity Extraction
  import nltk
  from nltk import word_tokenize,pos_tag
  label={'date':1,'gpe':2,'cardinal':0}
  lh=['0','0','0','0']
  nam=[]
  h=0
  for sent in nltk.sent_tokenize(sen):
    d=nltk.word_tokenize(sent)
    c=0
    for chunk in nltk.ne_chunk(nltk.pos_tag(d)):
      if hasattr(chunk, 'label'):
        if(chunk.label()=='GPE') and h<2:
          lh[label[chunk.label().lower()]+h]=d[c]
          h+=1
        elif chunk.label()=='PERSON' and len(nam)<=int(lh[0]):
          nam.append(d[c])
        elif chunk.label().lower() in label:
          lh[label[chunk.label().lower()]]=d[c]
      c+=1
  return ticketfunc(lh,nam,len(nam))

In [ ]:
def exp(inp,c):
  imp='chennai|trichy|madurai|kerala|vilupuram'
  mnt='january|febraury|march|april|may|june|july|august|september|october|november|december'
  try:
    if(c==0):
      q=re.findall(r'(\d+)',inp.strip())
      p=q[0]
    elif(c==1):
      q=re.findall(imp,inp.strip().lower())
      p=q[0]
    elif(c==2):
      p=''
      d=re.findall(r'(\d+)',inp.strip())
      m=re.findall(mnt,inp.strip().lower())
      for i in d:
        p=p+i+' '
      for i in m:
        p=p+i
    return p
  except IndexError:
    return '0'

In [ ]:
def ticketfunc(lh,nam,cnt):
  p=["How many ticket","Which date for boarding","What is the starting point","What is the destination","What is the passenger name"]
  #preprocessing
  #lhb = p
  while(True):
    if lh[0]=='0':
      result=p[0]
      i=0
      c=0
    elif lh[1]=='0':
      result=p[1]
      i=1
      c=2
    elif lh[2]=='0':
      result=p[2]
      i=2
      c=1
    elif lh[3]=='0':
      result=p[3]
      i=3
      c=1
    elif cnt<int(lh[0]):
      result=p[4]
      translated = translator.translate(result,src='en',dest='ta')
      print(translated.text)
      nam.append(audio_input())
      cnt+=1
      continue
    else:
      break
    translated = translator.translate(result,src='en',dest='ta')
    print(translated.text)
    lh[i]=exp(audio_input(),c)
  return lh, nam

In [ ]:
import time

In [ ]:
print("வணக்கம், நான் உங்கள் தனிப்பட்ட வழிகாட்டி ஓமி")
while(True):
  record = input().lower()
  if record == "y":
    sen = audio_input()
    if sen == 0:
      break
    l1 = ["ticket","train"]
    flag=0
    for i in l1:
      if i in sen:
        flag=1
    if flag==1:
      p,name=func(sen)
      print("நன்றி விரைவில் சந்திப்போம்")
      break
    else:
      msg = sendmessage(sen)
      print(msg)
  elif record=="stop":
    break
  else:
    time.sleep(10)

வணக்கம், நான் உங்கள் தனிப்பட்ட வழிகாட்டி ஓமி
y


எப்படி இருக்க
வணக்கம்
y


உன் பெயர் என்ன
என் பெயர் ஓமி
y


எனக்கு டிக்கெட் வேணுமா திருச்சியில் இருந்து சென்னை போகும் போது
எத்தனை டிக்கெட்


2
ஏறுவதற்கு எந்த தேதி


30aug
இலக்கு என்ன


சென்னை
பயணியின் பெயர் என்ன


ஓம் பிரகாஷ்
பயணியின் பெயர் என்ன


அஸ்வின்
நன்றி விரைவில் சந்திப்போம்


In [ ]:
p

['2', '௩௦ ', 'Trichy', 'chennai']

In [ ]:
name

['Om Prakash', 'Ashwin']